<a href="https://colab.research.google.com/github/DaviAmerico/project1/blob/main/Competi%C3%A7%C3%A3o_House_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resolução da competição House Prices - Advanced Regression Techniques
Nosso objetivo aqui é criar um modelo que preveja os valores de determinadas casas com base em 79 variáveis explorátorias, deveremos fazer isso usando um modelo de regressão.A resolução dessa competição se dará em três tópicos: **Limpeza dos dados**,**Transformando os dados** e **Criando o modelo e prevendo os preços**.



**Limpeza dos dados**:




Nosso objetivo nessa seção é tratar os dados faltantes(*nan's*), mas antes vamos chamar os dados a serem usados.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from scipy import stats
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
train=pd.read_csv('C:/Users/IWFY/Desktop/competição kaggle/train.csv')
test=pd.read_csv('C:/Users/IWFY/Desktop/competição kaggle/test.csv')

Nas sete primeiras linhas estamos importando os pacotes a serem usados , nas duas últimas estamos guardando em *train* os dados que serão usados para criar o modelo , em *test* estamos guardando os dados que serão usados para prever os preços.

Agora vamos procurar dados faltantes nos dois conjuntos de dados:



In [ ]:
train.isna().sum().sum()
test.isna().sum().sum()

1

No conjunto *train* temos 6965 valores faltantes e no conjunto *test* temos 7000.Inicialmente vamos tratar os *nan's* nas colunas númericas:

In [ ]:
train.fillna(train.mean(),inplace=True)
test.fillna(test.mean(),inplace=True)

Acima substituimos os valores faltantes das colunas númericas pela média dos valores em cada coluna.Pelo fato de termos usado a média as colunas categóricas não foram alteradas.

Agora vamos tratar os dados faltantes categóricos, infelizmente não é tão simples quanto para dados númericos , teremos que usar uma recursividade para substituí-los:

In [ ]:
for i in train.select_dtypes('object').columns:
    train.loc[:,i].fillna(train.loc[:,i].value_counts().index[0],inplace=True)
    test.loc[:,i].fillna(test.loc[:,i].value_counts().index[0],inplace=True)

Acima estamos percorrendo *i* nas colunas categóricas dos dados , para cada coluna os *nan's* serão substituidos pela categoria mais frequente da respectiva coluna.

Podemos ver então que não existem mais valores faltante nos dois conjuntos de dados:

In [ ]:
train.isna().sum().sum()
test.isna().sum().sum()

**Transformando os dados:**

A maioria dos algoritmos de aprendizado de maquina só aceitam dados númericos e funcionam melhor com dados normalizados.Abaixo vamos transformar todas categorias em números:

In [ ]:
for i in train.select_dtypes('object').columns:
    train.loc[:,i]=LabelEncoder().fit_transform(train.loc[:,i].astype('str'))
    test.loc[:,i]=LabelEncoder().fit_transform(test.loc[:,i].astype('str'))

Podemos ver agora que cada coluna dos dois conjuntos é númerica:

In [ ]:
train.dtypes
test.dtypes

Antes de normalizar os dados vamos separar a variável resposta das explicativas , pois estamos interessados em apenas normalizar as explicativas:

In [ ]:
xt=train.iloc[:,0:80]
yt=train.iloc[:,80]
xte=test


Em *xt* e *xte* temos as variáveis explicativas a serem normalizadas nos conjuntos *train* e *test*, note que *xte* é igual a *test*,pois o conjunto de dados *test* não contém observações da variável resposta , até porque queremos prevê-las para essa conjunto. 

Normalizando os dados, exceto a resposta, temos:

In [ ]:
zt=pd.DataFrame(StandardScaler().fit_transform(xt))
zt.columns=xt.columns
zte=pd.DataFrame(StandardScaler().fit_transform(xte))
zte.columns=xte.columns

*zt* e *zte* são os preditores normalizados para ambos os conjuntos de dados.

Agora estamos preparados para criar nosso modelo e fazer as predições :D

**Criando o modelo e prevendo os preços:**

O modelo a ser usado para resolver o problema de regressão é o *XGBoosting(Extreme Gradient Boosting)*, e ele será treinado em *(zt,yt)*.Iremos testar várias combinações aleatórias de hiper-parâmetros nele e pegar a combinação ótima depois de um determinado número de tentativas,no caso são 25 tentativas, para nós a combinação ótima é aquela que obteve a menor raiz do erro quadrático logarítmico médio,métrica de avaliação usada na competição, dentro da validação cruzada com 5 *folds*,então:

In [ ]:
xgbr=xgb.XGBRegressor()
param_grid = {'n_estimators': stats.randint(150, 500),
              'learning_rate': stats.uniform(0.01, 0.07),
              'subsample': stats.uniform(0.3, 0.7),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.45,0.5),
              'min_child_weight': [1, 2, 3]
             }
f=RandomizedSearchCV(xgbr,param_grid,cv=5,n_iter=25,scoring='neg_mean_squared_log_error')
f.fit(zt,yt)

Podemos,também, verificar o erro da melhor combinação de hiper-parâmetros:

In [ ]:
bs=np.sqrt(-f.best_score_)

Agora podemos fazer a previsão dos preços das casas dentro do conjunto *test* normalizado:

In [ ]:
yp=f.best_estimator_.predict(zte)

Agora basta exportar as predições para o Kaggle para concluir a competição :D